In [1]:
import pandas as pd
import numpy as np
from score_submission import our_scorer, scorer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn import datasets, linear_model
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor
import pickle

In [2]:
from functions.date_split import split_month_test
from score_submission import our_scorer, scorer
#import functions.CreatingNewDataframe as CreatingNewDataframe
import functions.GenerateFis as Features
import functions.GenerateMeansWithOffset as MeansOffset
from functions.pipeline_steps import *

In [3]:
with open("model_to_run.pickle", "rb") as file:
    clf = pickle.load(file)

## Классификатор обучен

In [4]:
df_test = pd.read_csv('train.csv.zip', parse_dates=['DATE'])
ids = np.unique(df_test.ATM_ID)

### Добавим даты для predict

In [5]:
pred_dates  = np.array(['2017-08-16', '2017-08-17', '2017-08-18', '2017-08-19',
               '2017-08-20', '2017-08-21', '2017-08-22', '2017-08-23',
               '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27',
               '2017-08-28', '2017-08-29', '2017-08-30', '2017-08-31',
               '2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04',
               '2017-09-05', '2017-09-06', '2017-09-07', '2017-09-08',
               '2017-09-09', '2017-09-10', '2017-09-11', '2017-09-12',
               '2017-09-13', '2017-09-14', '2017-09-15', '2017-09-16',
               '2017-09-17'], dtype='datetime64')

data = []
for id in ids:
    for date in pred_dates:
        data.append((date, id, np.nan))
df_test_append = pd.DataFrame(data=data, columns=['DATE', 'ATM_ID', 'CLIENT_OUT'])
df_test = df_test.append(df_test_append)

### Создадим фичи

In [6]:
df1, df2, df3, df4, df5 = Features.GenerateFis(df_test)

In [8]:
test_days = len(pred_dates)
#_, test_y = split_month_test(df_initial, train_days) # для теста данных нет
_, test1_X = split_month_test(df1, test_days)

# Получим ответ

In [9]:
test_y_pred = clf.predict(test1_X)

WrapperForEstimator transform called
Transform from estimator Finished
300
FinalStepForSubmission Finished


In [13]:
test_y_pred.where(data_frame < 0, 0)

,DATE,ATM_ID,CLIENT_OUT
0,2017-08-16,0,402337.902710
1,2017-08-17,0,455569.891743
2,2017-08-18,0,464167.553689
3,2017-08-19,0,170642.965800
4,2017-08-20,0,157753.903910
5,2017-08-21,0,409613.325381
6,2017-08-22,0,334676.080299
7,2017-08-23,0,301737.784366
8,2017-08-24,0,403739.001660
9,2017-08-25,0,446934.450750


In [12]:
test_y_pred.to_csv('submission.csv', index=False)